Advanced: Settings
============================

The Bundle also contains a few Parameters that provide settings for that Bundle.  Note that these are not system-wide and only apply to the current Bundle.  They are however maintained when [saving and loading](./saving_and_loading.ipynb) a Bundle.

Setup
-----------------------------

Let's first make sure we have the latest version of PHOEBE 2.4 installed (uncomment this line if running in an online notebook session such as colab).

In [1]:
#!pip install -I "phoebe>=2.4,<2.5"

As always, let's do imports and initialize a longger and a new Bundle.

In [2]:
import phoebe
from phoebe import u # units

logger = phoebe.logger()

b = phoebe.default_binary()

Accessing Settings
---------------------------

Settings are found with their own context in the Bundle and can be accessed through the get_setting method

In [3]:
b.get_setting()

<ParameterSet: 11 parameters | qualifiers: auto_remove_figure, dict_filter, run_checks_compute, run_checks_solver, auto_add_figure, dict_set_all, run_checks_solution, web_client, web_client_url, phoebe_version, run_checks_figure>

or via filtering/twig access

In [4]:
b['setting']

<ParameterSet: 11 parameters | qualifiers: auto_remove_figure, dict_filter, run_checks_compute, run_checks_solver, auto_add_figure, dict_set_all, run_checks_solution, web_client, web_client_url, phoebe_version, run_checks_figure>

and can be set as any other Parameter in the Bundle

Available Settings
--------------------------

Now let's look at each of the available settings and what they do

### phoebe_version

`phoebe_version` is a read-only parameter in the settings to store the version of PHOEBE used.

### dict_set_all

`dict_set_all` is a BooleanParameter (defaults to False) that controls whether attempting to set a value to a ParameterSet via dictionary access will set all the values in that ParameterSet (if True) or raise an error (if False)

In [5]:
b['dict_set_all@setting']

<Parameter: dict_set_all=False | keys: description, value, visible_if, copy_for, readonly, advanced, latexfmt>

In [6]:
b['teff@component']

<ParameterSet: 2 parameters | components: primary, secondary>

In our default binary there are temperatures ('teff') parameters for each of the components ('primary' and 'secondary').  If we were to do:

b['teff@component'] = 6000

this would raise an error.  Under-the-hood, this is simply calling:

b.set_value('teff@component', 6000)

which of course would also raise an error.

In order to set both temperatures to 6000, you would either have to loop over the components or call the set_value_all method:

In [7]:
b.set_value_all('teff@component', 4000)
print(b['value@teff@primary@component'], b['value@teff@secondary@component'])

4000.0 4000.0


If you want dictionary access to use set_value_all instead of set_value, you can enable this parameter

In [8]:
b['dict_set_all@setting'] = True
b['teff@component'] = 8000
print(b['value@teff@primary@component'], b['value@teff@secondary@component'])

Wed, 16 Sep 2020 22:04 BUNDLE       WARNING 'primary' probably has a radiative atm (teff=8000K>8000K), for which gravb_bol>=0.9 might be a better approx than gravb_bol=0.32.
Wed, 16 Sep 2020 22:04 BUNDLE       WARNING 'primary' probably has a radiative atm (teff=8000K>=8000K), for which irrad_frac_refl_bol>0.8 (suggestion: 1.0) might be a better approx than irrad_frac_refl_bol=0.60.
Wed, 16 Sep 2020 22:04 BUNDLE       WARNING 'primary' probably has a radiative atm (teff=8000K>8000K), for which gravb_bol>=0.9 might be a better approx than gravb_bol=0.32.
Wed, 16 Sep 2020 22:04 BUNDLE       WARNING 'secondary' probably has a radiative atm (teff=8000K>8000K), for which gravb_bol>=0.9 might be a better approx than gravb_bol=0.32.
Wed, 16 Sep 2020 22:04 BUNDLE       WARNING 'primary' probably has a radiative atm (teff=8000K>=8000K), for which irrad_frac_refl_bol>0.8 (suggestion: 1.0) might be a better approx than irrad_frac_refl_bol=0.60.
Wed, 16 Sep 2020 22:04 BUNDLE       WARNING 'seconda

8000.0 8000.0


Now let's disable this so it doesn't confuse us while looking at the other options

In [9]:
b.set_value_all('teff@component', 6000)
b['dict_set_all@setting'] = False

Wed, 16 Sep 2020 22:04 BUNDLE       WARNING 'secondary' probably has a radiative atm (teff=8000K>8000K), for which gravb_bol>=0.9 might be a better approx than gravb_bol=0.32.
Wed, 16 Sep 2020 22:04 BUNDLE       WARNING 'secondary' probably has a radiative atm (teff=8000K>=8000K), for which irrad_frac_refl_bol>0.8 (suggestion: 1.0) might be a better approx than irrad_frac_refl_bol=0.60.


### dict_filter

`dict_filter` is a Parameter that accepts a dictionary.  This dictionary will then always be sent to the filter call which is done under-the-hood during dictionary access.

In [10]:
b['incl']

<ParameterSet: 5 parameters | contexts: constraint, component>

In our default binary, there are several inclination parameters - one for each component ('primary', 'secondary', 'binary') and one with the constraint context (to keep the inclinations aligned).

This can be inconvenient... if you want to set the value of the binary's inclination, you must always provide extra information (like '@component').

Instead, we can always have the dictionary access search in the component context by doing the following

In [11]:
b['dict_filter@setting'] = {'context': 'component'}

In [12]:
b['incl']

<ParameterSet: 3 parameters | kinds: star, orbit>

Now we no longer see the constraint parameters.

All parameters are always accessible with method access:

In [13]:
b.filter(qualifier='incl')

<ParameterSet: 5 parameters | contexts: constraint, component>

Now let's reset this option... keeping in mind that we no longer have access to the 'setting' context through twig access, we'll have to use methods to clear the dict_filter

In [14]:
b.set_value('dict_filter@setting', {})

### run_checks_compute (/figure/solver/solution)

The `run_checks_compute` option allows setting the default compute option(s) sent to [b.run_checks](../api/phoebe.frontend.bundle.Bundle.run_checks.md), including warnings in the logger raised by interactive checks (see [phoebe.interactive_checks_on](../api/phoebe.interactive_checks_on.md)).

Similar options also exist for checks at the figure, solver, and solution level.

In [15]:
b['run_checks_compute@setting']

<Parameter: run_checks_compute=['*'] | keys: description, choices, value, visible_if, readonly, copy_for, latexfmt>

In [16]:
b.add_dataset('lc')
b.add_compute('legacy')
print(b.run_checks())

Run Checks Report: FAIL
ERROR: ld_mode='interp' not supported by 'legacy' backend used by compute='legacy01'.  Change ld_mode@primary@lc01. (2 affected parameters, affecting run_compute)
ERROR: ld_mode='interp' not supported by 'legacy' backend used by compute='legacy01'.  Change ld_mode@secondary@lc01. (2 affected parameters, affecting run_compute)


In [17]:
b['run_checks_compute@setting'] = ['phoebe01']

In [18]:
print(b.run_checks())

Run Checks Report: PASS



### auto_add_figure, auto_remove_figure

The `auto_add_figure` and `auto_remove_figure` determine whether new figures are automatically added to the Bundle when new datasets, distributions, etc are added.  This is False by default within Python, but True by default within the [UI clients](http://phoebe-project.org/clients).

In [19]:
b['auto_add_figure']

<Parameter: auto_add_figure=True | keys: description, value, visible_if, copy_for, readonly, advanced, latexfmt>

In [20]:
b['auto_add_figure'].description

'Whether to automatically add figure parameters when a dataset is added with a new dataset type, or a solution is added.'

In [21]:
b['auto_remove_figure']

<Parameter: auto_remove_figure=True | keys: description, value, visible_if, copy_for, readonly, advanced, latexfmt>

In [22]:
b['auto_remove_figure'].description

'Whether to automatically remove figure parameters when the referenced dataset/solution are removed.'

### web_client, web_client_url

The `web_client` and `web_client_url` settings determine whether the [client](http://phoebe-project.org/clients) is opened in a web-browser or with the installed desktop client whenever calling [b.ui](../api/phoebe.parameters.ParameterSet.ui.md) or [b.ui_figures](../api/phoebe.frontend.bundle.Bundle.ui_figures.md).  For more information, see the [UI from Jupyter tutorial](./ui_jupyter.ipynb).

In [23]:
b['web_client']

<Parameter: web_client=False | keys: description, value, visible_if, copy_for, readonly, advanced, latexfmt>

In [24]:
b['web_client'].description

'Whether to default to using the web-client over a locally installed desktop-client when opening the UI from the desktop client.'

In [25]:
b['web_client_url']

<Parameter: web_client_url=ui.phoebe-project.org | keys: description, value, visible_if, copy_for, readonly, advanced, latexfmt>

In [26]:
b['web_client_url'].description

'Default location of web-client.  Will only be used if web_client is True.'